## Day 12
https://adventofcode.com/2025/day/12

In [1]:
import numpy as np
import time

In [2]:
def read_input_12(filename):
    with open(filename) as f:
        situation = f.read().split("\n\n")
        shapes = [np.array([[1 if c=="#" else 0 for c in r] for r in b.split("\n")[1:] ]) for b in situation[:-1]]
        regions = []
        for r in situation[-1].strip("\n").split("\n"):
            t = r.split(": ")
            size = tuple([int(i) for i in t[0].split("x")])
            tiles = [int(i) for i in t[1].split(" ")]
            regions.append((size, tiles))
    return shapes, regions

In [45]:
def all_rotations_and_flips(mat):
    """Return unique transformations of a 2D matrix."""
    transforms = []
    
    # 4 rotations
    for k in range(4):
        transforms.append(np.rot90(mat, k))
    
    # Flipped versions
    flipped = np.flip(mat, axis=1)
    for k in range(4):
        transforms.append(np.rot90(flipped, k))
    
    # Remove duplicates by converting to tuples
    unique = []
    seen = set()
    for t in transforms:
        key = t.tobytes()
        if key not in seen:
            seen.add(key)
            unique.append(t)
    
    return unique

def precompute_placements(board_shape, shape_variants):
    """
    Precompute all valid (r, c) positions where each shape variant can be placed.
    Returns list of (variant_idx, row, col, positions_array) tuples.
    positions_array is a list of (row, col) coordinates where the shape has a 1.
    """
    nr, nc = board_shape
    placements = []
    for v_idx, variant in enumerate(shape_variants):
        vr, vc = variant.shape
        # Get positions of 1 in the variant
        positions = [(r, c) for r in range(vr) for c in range(vc) if variant[r, c] == 1]
        for r in range(nr - vr + 1):
            for c in range(nc - vc + 1):
                # Translate positions to board coordinates
                board_positions = [(r + pr, c + pc) for pr, pc in positions]
                placements.append((v_idx, r, c, board_positions)) 
    return placements

def can_place(board, positions):
    """Check if all positions are free on the board."""
    return all(board[r, c] == 0 for r, c in positions)

def place(board, positions, value=1):
    """Place a shape on the board."""
    for r, c in positions:
        board[r, c] = value

def unplace(board, positions):
    """Remove a shape from the board."""
    for r, c in positions:
        board[r, c] = 0

START_TIME = None
TIME_LIMIT = 2.0 # seconds

def backtrack(board, shape_placements, remaining_shapes, time_cut=False):
    """
    Backtracking with in-place board modifications
    - board: numpy array representing the current board state
    - shape_placements: dict mapping shape_idx -> list of precomputed placements
    - remaining_shapes: list of shape indices still to place
    """
    if not remaining_shapes:
        return True

    if time_cut and time.time() - START_TIME > TIME_LIMIT:
        return False # abort early
    
    # Pick next shape to place
    shape_idx = remaining_shapes[0]
    placements = shape_placements[shape_idx]
    
    # Try each placement
    for v_idx, r, c, positions in placements:
        if can_place(board, positions):
            # Place the shape
            place(board, positions)
            
            # Recurse
            if backtrack(board, shape_placements, remaining_shapes[1:], time_cut):
                return True
            
            # Backtrack: remove the shape
            unplace(board, positions)
    
    return False

def solve_region(region, shapes, check_region_size=True, time_limit=0):
    """Solve a region using backtracking"""

    global START_TIME, TIME_LIMIT
    time_cut = False
    if time_limit>0:
        TIME_LIMIT = time_limit
        START_TIME = time.time()
        time_cut = True

    size, tiles = region
    nr, nc = size
    
    # Build list of shape indices to place
    shape_list = []
    for i, count in enumerate(tiles):
        shape_list.extend([i] * count)

    # Check whether shape size is bigger than region area
    # I thought about implementing this check while drivingg to work, but then forgot about it!
    # And actually it fixes all the regions with no solution (damn Eric!)
    if check_region_size:
        if sum(sum(sum([shapes[i] for i in shape_list]))) > size[0]*size[1]:
            return False
    
    # Sort shapes by size (larger first) by counting filled cells for each shape
    # It should helps to prune search space during backtracking (hopeless solutions are reached earlier)
    shape_sizes = [(i, np.sum(shapes[i])) for i in range(len(shapes))]
    shape_list.sort(key=lambda idx: shape_sizes[idx][1], reverse=True)
    
    # Precompute all transformations for each unique shape
    shape_variants = {}
    for shape_idx in set(shape_list):
        shape_variants[shape_idx] = all_rotations_and_flips(shapes[shape_idx])
    
    # Precompute all valid placements for each shape
    shape_placements = {}
    for shape_idx in set(shape_list):
        shape_placements[shape_idx] = precompute_placements((nr, nc), shape_variants[shape_idx])
    
    # Initialize board
    board = np.zeros((nr, nc), dtype=np.int8)
    
    return backtrack(board, shape_placements, shape_list, time_cut)

def part1(filename, verbose=True, check_region_size=False, time_limit=0):

    start_time = time.time()    
    shapes, regions = read_input_12(filename)
    count = 0
    
    for i, region in enumerate(regions):
        region_start = time.time()
        solved = solve_region(region, shapes, check_region_size, time_limit)
        region_time = time.time() - region_start
        
        if verbose: 
            print(f"Region {i+1}: {region[0]} with tiles {region[1]} -> {solved} (took {region_time:.3f}s)")
        else:
            print(f"Region {i+1} / {len(regions)}",end="\r")
        if solved:
            count += 1
    
    print(f"\nRegions that can fit selected shapes: {count}")
    total_time = time.time() - start_time        
    print(f"Total execution time: {total_time:.3f}s")
    
    return count

In [20]:
part1("examples/example12.txt", verbose=True)

Region 1: (4, 4) with tiles [0, 0, 0, 0, 2, 0] -> True (took 0.000s)
Region 2: (12, 5) with tiles [1, 0, 1, 0, 2, 2] -> True (took 0.003s)
Region 3: (12, 5) with tiles [1, 0, 1, 0, 3, 2] -> False (took 97.731s)

Regions that can fit selected shapes: 2
Total execution time: 97.738s


2

If a solution exists, it is found rather quickly. 

On the other hand, **if a solution does not exist, backtracking takes a long time to explore the full phase space (and there are 1000 large regions with lots of shapes to be tested in the full input!).** The correct solution to this would be to implement pruning strategies, but before attempting it **I will try to simply top the solving procedure that takes too long to converge ;-)**. 

This is potentially dangerous, since it does not guarantee that a solution could ultimately be found in longer searches...

In [44]:
part1("examples/example12.txt", verbose=True, time_limit=2)

Region 1: (4, 4) with tiles [0, 0, 0, 0, 2, 0] -> True (took 0.000s)
Region 2: (12, 5) with tiles [1, 0, 1, 0, 2, 2] -> True (took 0.002s)
Region 3: (12, 5) with tiles [1, 0, 1, 0, 3, 2] -> False (took 2.000s)

Regions that can fit selected shapes: 2
Total execution time: 2.004s


2

In [47]:
part1("AOC2025inputs/input12.txt", verbose=False, time_limit=1)

Region 1000 / 1000
Regions that can fit selected shapes: 575
Total execution time: 3285.280s


575

**... and the dirty approach works!!! ;-)**

The simplest pruning strategy I can try is to check whether **there's enough space in the region to contain all shapes**. Not very effective for the example...

In [22]:
part1("examples/example12.txt", verbose=True, check_region_size=True)

Region 1: (4, 4) with tiles [0, 0, 0, 0, 2, 0] -> True (took 0.000s)
Region 2: (12, 5) with tiles [1, 0, 1, 0, 2, 2] -> True (took 0.003s)
Region 3: (12, 5) with tiles [1, 0, 1, 0, 3, 2] -> False (took 98.166s)

Regions that can fit selected shapes: 2
Total execution time: 98.170s


2

In [48]:
part1("AOC2025inputs/input12.txt", verbose=False, check_region_size=True)

Region 1000 / 1000
Regions that can fit selected shapes: 575
Total execution time: 64.405s


575

**... but it works great for the input: all regions with no solution seem to be such becouse of lack of space, not of possible combinations! Damn, Eric! ;-)**